In [28]:
!pip install geopandas
!pip install geoplot
!pip install -U kaleido

     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 16.7 MB 322 kB/s 
     |████████████████████████████████| 6.3 MB 38.0 MB/s 
     |████████████████████████████████| 10.8 MB 9.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 40.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 30.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 12.1 MB 21.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.3 MB 1.3 MB/s 
     |████████████████████████████████| 44 kB 1.5 MB/s 
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.wh

In [1]:
import os
import sys
import pandas as pd

In [2]:
#@title ## Mount google drive and import my code

mountpoint_folder_name = "gdrive"  # can be anything, doesn't have to be "drive"
project_path_within_drive = "PythonProjects/GeospatialAnalysis" #@param {type:"string"}

try:
    import google.colab.drive
    import os, sys
    # Need to move out of google drive directory if going to remount
    %cd
    # drive.mount documentation can be accessed via: drive.mount?
    #Signature: drive.mount(mountpoint, force_remount=False, timeout_ms=120000, use_metadata_server=False)
    google.colab.drive.mount(os.path.join("/content/",mountpoint_folder_name), force_remount=True)  # mounts to a folder called mountpoint_folder_name
    project_path_full = os.path.join("/content/",mountpoint_folder_name,
                            "MyDrive",project_path_within_drive)
    if project_path_full not in sys.path:
        pass
        #sys.path.insert(0,project_path_full)
    %cd {project_path_full}
    
except ModuleNotFoundError:  # in case not run in Google colab
    import traceback
    traceback.print_exc()

/root
Mounted at /content/gdrive
/content/gdrive/MyDrive/Computer Backups/Rahul Yerrabelli drive/PythonProjects/GeospatialAnalysis


In [29]:
!git clone https://github.com/ryerrabelli/GeospatialAnalysis.git

project_path_full = os.path.join("/content/","GeospatialAnalysis")
sys.path.insert(1,project_path_full)
!cd GeospatialAnalysis
print(sys.path)

fatal: destination path 'GeospatialAnalysis' already exists and is not an empty directory.


AttributeError: ignored

In [21]:
import pandas as pd
fips2county = pd.read_csv("data/fips2county.tsv", sep="\t", comment='#', dtype=str)
df_orig = pd.read_csv("data/2022_04_10 ent initial output.csv", dtype={"FIPS": str})
df = pd.merge(left=df_orig, right=fips2county, how="outer", left_on='FIPS', right_on='CountyFIPS')
df.insert(1, "County_St", df["CountyName"].astype(str) + ", " + df["StateAbbr"].astype(str))
with pd.option_context('display.max_rows', 3, 'display.max_columns', None): 
    display(df.head())
    #display(fips2county.head())

#display(df.columns)


,FIPS,County_St,Total Number of Services,Total Medicare Payment Amount,Total Number of Services: 2019,Total Medicare Payment Amount: 2019,Total Number of Services: 2018,Total Medicare Payment Amount: 2018,Total Number of Services: 2017,Total Medicare Payment Amount: 2017,Total Number of Services: 2016,Total Medicare Payment Amount: 2016,Total Number of Services: 2015,Total Medicare Payment Amount: 2015,tot_ratio,% ASC Procedures: 2019,% ASC Billing: 2019,% ASC Procedures: 2018,% ASC Billing: 2018,% ASC Procedures: 2017,% ASC Billing: 2017,% ASC Procedures: 2016,% ASC Billing: 2016,% ASC Procedures: 2015,% ASC Billing: 2015,% ASC Procedures,% ASC Billing,Beneficiaries with Part A and Part B,Average Age,Percent Male,Percent Non-Hispanic White,Percent African American,Percent Hispanic,Percent Eligible for Medicaid,Average HCC Score,Hospital Readmission Rate,Emergency Department Visits per 1000 Beneficiaries,Procedures Per Capita Standardized Costs,Procedure Events Per 1000 Beneficiaries,metro,pct_poverty,median_house_income,pop,2013_Rural_urban_cont_code,Pct_wthout_high_diploma,Pct_wth_high_diploma,Pct_wth_some_coll,Pct_wth_coll_degree,unemployment,pct_uninsured,fibro,tabacco,obesity,migrane,Alzheimers,Depression,Alcohol Abuse,Drug Abuse,Schizo_othr_psych,COPD,Chronic Kidney Disease,Osteoporosis,Stroke,Diabetes,Asthma,Arthritis,Hypertension,Heart Failure,Ischemic Heart Disease,Population Density,Medicare Population Density,Moran I score for ACS billing fraction,County,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY
0,01017,"Chambers, AL",408.0,30064.80,157.0,10363.09,115.0,6360.51,136.0,13341.2,0.0,0.0,0.0,0.00,14.196990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8489.0,69.2,44.126,64.466,34.326,0.286,28.144,1.040,16.296,726.4,502.664,3765.2,0.0,19.76,38781.0,33660.4,6.0,18.4,36.7,31.6,13.3,4.52,11.94,23.6,11.0,22.2,2.8,11.06,13.92,2.125,3.500,3.30,14.46,25.76,4.40,3.54,34.7,4.10,35.56,67.00,16.78,31.58,56.426908,14.230610,Non Significant,Chambers,01,017,Chambers,Alabama,01017,AL,AL | CHAMBERS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,05145,"White, AR",1269.0,52190.22,0.0,0.00,15.0,989.45,22.0,1663.2,15.0,1125.0,1217.0,48412.57,11.995594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15758.0,70.6,44.814,95.510,2.446,0.918,20.582,0.942,17.884,695.8,598.714,4411.4,0.0,16.52,44424.8,78762.0,4.0,15.6,39.3,24.2,20.9,4.92,9.54,22.2,12.2,16.6,3.4,12.72,18.32,1.300,3.725,3.48,14.82,22.02,4.84,4.06,26.5,3.64,33.32,61.72,14.10,33.24,76.093037,15.224018,Non Significant,White,05,145,White,Arkansas,05145,AR,AR | WHITE


In [24]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


import plotly.express as px

fig = px.choropleth(df, geojson=counties, locations='FIPS', color='tot_ratio',
                    color_continuous_scale="Viridis",
                    range_color=(0, 12),
                    scope="usa",
                    labels={
                        "2013_Rural_urban_cont_code":"2013-RUCA",
                        "pop":"Pop.",
                        "Average Age":"Mean Age",
                        "Percent Male":"% M.",
                        "tot_ratio":"Tot. Ratio",
                        },
                    hover_name="County_St",
                    hover_data={
                        "FIPS":True, 
                        "pop":True, 
                        "2013_Rural_urban_cont_code":True,
                        "Average Age":True, 
                        "Percent Male":True},
                    )

# Define layout specificities
fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    title={
        'text': f"Title",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [9]:
if not os.path.exists("images"):
    os.mkdir("images")

fig.write_image("images/fig1.svg")
fig.write_image("images/fig1.png")
fig.write_image("images/fig1.svg")
fig.write_html("images/fig1.html")


In [8]:
os.getcwd()
os.listdir()

['src',
 'versioneer.py',
 'MANIFEST.in',
 '.idea',
 'venv',
 '.git',
 'docs',
 'dist',
 'build',
 'tests',
 'examples',
 'lib',
 'data',
 '.gitignore',
 '.gitattributes',
 '__pycache__',
 'setup.cfg',
 'setup.py',
 'requirements.txt',
 'images']